<a href="https://colab.research.google.com/github/N2000-hack/Music_Genre_Classification_Project/blob/main/Dataset_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost -U
!pip install librosa -U
!pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)
import os
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV,mutual_info_regression
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset_onlynames_csv')
df.head()

,label
0,blues
1,blues
2,blues
3,blues
4,blues


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #1-8

def extract_new_features(song_path, num_files = 1000, num_new_features = 8):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))
                chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
                spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
                spectral_flatness = librosa.feature.spectral_flatness(y=y)
                tonnetz = librosa.feature.tonnetz(y=y, sr=sr)



                data_array[i,0] = np.mean(chroma_cens)
                data_array[i,1] = np.var(chroma_cens)
                data_array[i,2] = np.mean(spectral_contrast)
                data_array[i,3] = np.var(spectral_contrast)
                data_array[i,4] = np.mean(spectral_flatness)
                data_array[i,5] = np.var(spectral_flatness)
                data_array[i,6] = np.mean(tonnetz)
                data_array[i,7] = np.var(tonnetz)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:

df['chroma_cens_mean'] = new_features_array[:,0]
df['chroma_cens_var'] = new_features_array[:,1]
df['spectral_contrast_mean'] = new_features_array[:,2]
df['spectral_contrast_var'] = new_features_array[:,3]
df['spectral_flatness_mean'] = new_features_array[:,4]
df['spectral_flatness_var'] = new_features_array[:,5]
df['tonnetz_mean'] = new_features_array[:,6]
df['tonnetz_var'] = new_features_array[:,7]


In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69
...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #9-16

def extract_new_features(song_path, num_files = 1000, num_new_features = 8):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
                mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
                rms = librosa.feature.rms(y=y)

                data_array[i,0] = np.mean(chroma_stft)
                data_array[i,1] = np.var(chroma_stft)
                data_array[i,2] = np.mean(chroma_cq)
                data_array[i,3] = np.var(chroma_cq)
                data_array[i,4] = np.mean(mel_spec)
                data_array[i,5] = np.var(mel_spec)
                data_array[i,6] = np.mean(rms)
                data_array[i,7] = np.var(rms)



            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['chroma_stft_mean'] = new_features_array[:,0]
df['chroma_stft_var'] = new_features_array[:,1]
df['chroma_cq_mean'] = new_features_array[:,2]
df['chroma_cq_var'] = new_features_array[:,3]
df['mel_spec_mean'] = new_features_array[:,4]
df['mel_spec_var'] = new_features_array[:,5]
df['rms_mean'] = new_features_array[:,6]
df['rms_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,chroma_stft_var,chroma_cq_mean,chroma_cq_var,mel_spec_mean,mel_spec_var,rms_mean,rms_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914947e-310,6.914919e-310
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914943e-310,6.914943e-310,6.914906e-310,6.914906e-310
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #17-24

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                poly_feat = librosa.feature.poly_features(y=y, order=0)

                data_array[i,0] = np.mean(spec_cent)
                data_array[i,1] = np.var(spec_cent)
                data_array[i,2] = np.mean(spec_bw)
                data_array[i,3] = np.var(spec_bw)
                data_array[i,4] = np.mean(spec_rolloff)
                data_array[i,5] = np.var(spec_rolloff)
                data_array[i,6] = np.mean(poly_feat)
                data_array[i,7] = np.var(poly_feat)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['spec_cent_mean'] = new_features_array[:,0]
df['spec_cent_var'] = new_features_array[:,1]
df['spec_bw_mean'] = new_features_array[:,2]
df['spec_bw_var'] = new_features_array[:,3]
df['spec_rolloff_mean'] = new_features_array[:,4]
df['spec_rolloff_var'] = new_features_array[:,5]
df['poly_feat_mean'] = new_features_array[:,6]
df['poly_feat_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,rms_mean,rms_var,spec_cent_mean,spec_cent_var,spec_bw_mean,spec_bw_var,spec_rolloff_mean,spec_rolloff_var,poly_feat_mean,poly_feat_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,6.914947e-310,6.914919e-310,0.000000e+00,2.371515e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,6.914906e-310,6.914906e-310,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.124658e-312,1.145878e-312,1.124658e-312
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,6.914906e-310,6.914906e-310,2.470328e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,6.914906e-310,6.914906e-310,1.209538e-312,1.018558e-312,1.188318e-312,1.060998e-312,2.143216e-312,1.039778e-312,1.060998e-312,0.000000e+00
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,6.914906e-310,6.914906e-310,9.761181e-313,1.082218e-312,1.103438e-312,1.188318e-312,1.124658e-312,1.018558e-312,1.082218e-312,1.124658e-312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,6.914906e-310,6.914906e-310,1.167098e-312,1.167098e-312,1.167098e-312,1.039778e-312,1.145878e-312,9.548981e-313,1.039778e-312,0.000000e+00
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,6.914906e-310,6.914906e-310,9.761181e-313,1.082218e-312,1.103438e-312,1.188318e-312,1.124658e-312,1.018558e-312,1.209538e-312,1.039778e-312
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,6.914906e-310,6.914906e-310,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.209538e-312,1.060998e-312,1.018558e-312
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,6.914906e-310,6.914906e-310,2.143216e-312,1.060998e-312,1.060998e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #25-32

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                zero_cross_rate = librosa.feature.zero_crossing_rate(y)
                mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=10)
                mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
                mfccs2 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=30)


                data_array[i,0] = np.mean(zero_cross_rate)
                data_array[i,1] = np.var(zero_cross_rate)
                data_array[i,2] = np.mean(mfccs)
                data_array[i,3] = np.var(mfccs)
                data_array[i,4] = np.mean(mfccs1)
                data_array[i,5] = np.var(mfccs1)
                data_array[i,6] = np.mean(mfccs2)
                data_array[i,7] = np.var(mfccs2)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['zero_cross_rate_mean'] = new_features_array[:,0]
df['zero_cross_rate_var'] = new_features_array[:,1]
df['mfccs_mean'] = new_features_array[:,2]
df['mfccs_var'] = new_features_array[:,3]
df['mfccs1_mean'] = new_features_array[:,4]
df['mfccs1_var'] = new_features_array[:,5]
df['mfccs2_mean'] = new_features_array[:,6]
df['mfccs2_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,poly_feat_mean,poly_feat_var,zero_cross_rate_mean,zero_cross_rate_var,mfccs_mean,mfccs_var,mfccs1_mean,mfccs1_var,mfccs2_mean,mfccs2_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.188318e-312,1.103438e-312,1.103438e-312,1.167098e-312,1.188318e-312,1.209538e-312,1.188318e-312
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,1.145878e-312,1.124658e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.188318e-312,1.209538e-312,1.018558e-312
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,0.000000e+00,0.000000e+00,2.143216e-312,1.167098e-312,2.470328e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,1.060998e-312,0.000000e+00,1.124658e-312,1.167098e-312,1.082218e-312,1.060998e-312,2.143216e-312,1.060998e-312,1.060998e-312,0.000000e+00
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,1.082218e-312,1.124658e-312,9.761181e-313,1.103438e-312,1.060998e-312,1.103438e-312,1.209538e-312,1.167098e-312,1.039778e-312,1.103438e-312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,1.039778e-312,0.000000e+00,1.167098e-312,1.103438e-312,1.167098e-312,1.167098e-312,2.143216e-312,1.039778e-312,1.209538e-312,0.000000e+00
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,1.209538e-312,1.039778e-312,9.761181e-313,1.209538e-312,1.018558e-312,1.039778e-312,1.018558e-312,1.167098e-312,1.124658e-312,1.082218e-312
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,1.060998e-312,1.018558e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.103438e-312,1.124658e-312,1.082218e-312
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,0.000000e+00,0.000000e+00,1.145878e-312,9.548981e-313,1.145878e-312,2.519735e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #33-40

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                mfccs3 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
                mfccs4 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)
                mfccs5 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=60)
                mfccs6 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=70)



                data_array[i,0] = np.mean(mfccs3)
                data_array[i,1] = np.var(mfccs3)
                data_array[i,2] = np.mean(mfccs4)
                data_array[i,3] = np.var(mfccs4)
                data_array[i,4] = np.mean(mfccs5)
                data_array[i,5] = np.var(mfccs5)
                data_array[i,6] = np.mean(mfccs6)
                data_array[i,7] = np.var(mfccs6)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['mfccs3_mean'] = new_features_array[:,0]
df['mfccs3_var'] = new_features_array[:,1]
df['mfccs4_mean'] = new_features_array[:,2]
df['mfccs4_var'] = new_features_array[:,3]
df['mfccs5_mean'] = new_features_array[:,4]
df['mfccs5_var'] = new_features_array[:,5]
df['mfccs6_mean'] = new_features_array[:,6]
df['mfccs6_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,mfccs2_mean,mfccs2_var,mfccs3_mean,mfccs3_var,mfccs4_mean,mfccs4_var,mfccs5_mean,mfccs5_var,mfccs6_mean,mfccs6_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,1.209538e-312,1.188318e-312,0.000000e+00,2.371515e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,1.209538e-312,1.018558e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.124658e-312,1.145878e-312,1.124658e-312
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,0.000000e+00,0.000000e+00,2.470328e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,1.060998e-312,0.000000e+00,1.209538e-312,1.018558e-312,1.188318e-312,1.060998e-312,2.143216e-312,1.039778e-312,1.060998e-312,0.000000e+00
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,1.039778e-312,1.103438e-312,9.761181e-313,1.082218e-312,1.103438e-312,1.188318e-312,1.124658e-312,1.018558e-312,1.082218e-312,1.124658e-312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,1.209538e-312,0.000000e+00,1.167098e-312,1.167098e-312,1.167098e-312,1.039778e-312,1.145878e-312,9.548981e-313,1.039778e-312,0.000000e+00
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,1.124658e-312,1.082218e-312,9.761181e-313,1.082218e-312,1.103438e-312,1.188318e-312,1.124658e-312,1.018558e-312,1.209538e-312,1.039778e-312
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,1.124658e-312,1.082218e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.209538e-312,1.060998e-312,1.018558e-312
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,0.000000e+00,0.000000e+00,2.143216e-312,1.060998e-312,1.060998e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #41-48

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                mfccs7 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=15)
                mfccs8 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=25)
                mfccs9 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=35)
                mfccs10 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=45)


                data_array[i,0] = np.mean(mfccs7)
                data_array[i,1] = np.var(mfccs7)
                data_array[i,2] = np.mean(mfccs8)
                data_array[i,3] = np.var(mfccs8)
                data_array[i,4] = np.mean(mfccs9)
                data_array[i,5] = np.var(mfccs9)
                data_array[i,6] = np.mean(mfccs10)
                data_array[i,7] = np.var(mfccs10)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['mfccs7_mean'] = new_features_array[:,0]
df['mfccs7_var'] = new_features_array[:,1]
df['mfccs8_mean'] = new_features_array[:,2]
df['mfccs8_var'] = new_features_array[:,3]
df['mfccs9_mean'] = new_features_array[:,4]
df['mfccs9_var'] = new_features_array[:,5]
df['mfccs10_mean'] = new_features_array[:,6]
df['mfccs10_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,mfccs6_mean,mfccs6_var,mfccs7_mean,mfccs7_var,mfccs8_mean,mfccs8_var,mfccs9_mean,mfccs9_var,mfccs10_mean,mfccs10_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,1.145878e-312,1.124658e-312,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,0.000000e+00,0.000000e+00,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,1.060998e-312,0.000000e+00,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,1.082218e-312,1.124658e-312,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,1.039778e-312,0.000000e+00,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,1.209538e-312,1.039778e-312,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,1.060998e-312,1.018558e-312,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,0.000000e+00,0.000000e+00,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310,6.914906e-310


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #49-56

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                mfccs11 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=55)
                mfccs12 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=65)
                mfccs13 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=75)
                mfccs14 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=85)


                data_array[i,0] = np.mean(mfccs11)
                data_array[i,1] = np.var(mfccs11)
                data_array[i,2] = np.mean(mfccs12)
                data_array[i,3] = np.var(mfccs12)
                data_array[i,4] = np.mean(mfccs13)
                data_array[i,5] = np.var(mfccs13)
                data_array[i,6] = np.mean(mfccs14)
                data_array[i,7] = np.var(mfccs14)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['mfccs11_mean'] = new_features_array[:,0]
df['mfccs11_var'] = new_features_array[:,1]
df['mfccs12_mean'] = new_features_array[:,2]
df['mfccs12_var'] = new_features_array[:,3]
df['mfccs13_mean'] = new_features_array[:,4]
df['mfccs13_var'] = new_features_array[:,5]
df['mfccs14_mean'] = new_features_array[:,6]
df['mfccs14_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,mfccs10_mean,mfccs10_var,mfccs11_mean,mfccs11_var,mfccs12_mean,mfccs12_var,mfccs13_mean,mfccs13_var,mfccs14_mean,mfccs14_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,6.914906e-310,6.914906e-310,4.050624e-316,2.371515e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,6.914906e-310,6.914906e-310,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.124658e-312,1.145878e-312,1.124658e-312
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,6.914906e-310,6.914906e-310,2.470328e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,6.914906e-310,6.914906e-310,1.209538e-312,1.018558e-312,1.188318e-312,1.060998e-312,2.143216e-312,1.039778e-312,1.060998e-312,0.000000e+00
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,6.914906e-310,6.914906e-310,9.761181e-313,1.082218e-312,1.103438e-312,1.188318e-312,1.124658e-312,1.018558e-312,1.082218e-312,1.124658e-312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,6.914906e-310,6.914906e-310,1.167098e-312,1.167098e-312,1.167098e-312,1.039778e-312,1.145878e-312,9.548981e-313,1.039778e-312,0.000000e+00
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,6.914906e-310,6.914906e-310,9.761181e-313,1.082218e-312,1.103438e-312,1.188318e-312,1.124658e-312,1.018558e-312,1.209538e-312,1.039778e-312
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,6.914906e-310,6.914906e-310,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.209538e-312,1.060998e-312,1.018558e-312
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,6.914906e-310,6.914906e-310,2.143216e-312,1.060998e-312,1.060998e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #57-64

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                mfccs15 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=17)
                mfccs16 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=27)
                mfccs17 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=37)
                mfccs18 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=47)


                data_array[i,0] = np.mean(mfccs15)
                data_array[i,1] = np.var(mfccs15)
                data_array[i,2] = np.mean(mfccs16)
                data_array[i,3] = np.var(mfccs16)
                data_array[i,4] = np.mean(mfccs17)
                data_array[i,5] = np.var(mfccs17)
                data_array[i,6] = np.mean(mfccs18)
                data_array[i,7] = np.var(mfccs18)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['mfccs15_mean'] = new_features_array[:,0]
df['mfccs15_var'] = new_features_array[:,1]
df['mfccs16_mean'] = new_features_array[:,2]
df['mfccs16_var'] = new_features_array[:,3]
df['mfccs17_mean'] = new_features_array[:,4]
df['mfccs17_var'] = new_features_array[:,5]
df['mfccs18_mean'] = new_features_array[:,6]
df['mfccs18_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,mfccs14_mean,mfccs14_var,mfccs15_mean,mfccs15_var,mfccs16_mean,mfccs16_var,mfccs17_mean,mfccs17_var,mfccs18_mean,mfccs18_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.188318e-312,1.103438e-312,1.103438e-312,1.167098e-312,1.188318e-312,1.209538e-312,1.188318e-312
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,1.145878e-312,1.124658e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.188318e-312,1.209538e-312,1.018558e-312
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,0.000000e+00,0.000000e+00,2.143216e-312,1.167098e-312,2.470328e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,1.060998e-312,0.000000e+00,1.124658e-312,1.167098e-312,1.082218e-312,1.060998e-312,2.143216e-312,1.060998e-312,1.060998e-312,0.000000e+00
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,1.082218e-312,1.124658e-312,9.761181e-313,1.103438e-312,1.060998e-312,1.103438e-312,1.209538e-312,1.167098e-312,1.039778e-312,1.103438e-312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,1.039778e-312,0.000000e+00,1.167098e-312,1.103438e-312,1.167098e-312,1.167098e-312,2.143216e-312,1.039778e-312,1.209538e-312,0.000000e+00
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,1.209538e-312,1.039778e-312,9.761181e-313,1.209538e-312,1.018558e-312,1.039778e-312,1.018558e-312,1.167098e-312,1.124658e-312,1.082218e-312
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,1.060998e-312,1.018558e-312,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.761181e-313,1.103438e-312,1.124658e-312,1.082218e-312
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,0.000000e+00,0.000000e+00,1.145878e-312,9.548981e-313,1.145878e-312,2.519735e-322,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
songs_path = '/content/drive/MyDrive/genres_original' #65-72

def extract_new_features(song_path, num_files = 1000, num_new_features = 28):

    data_array = np.empty([num_files, num_new_features])

    counter = 0
    for root, dirs, files in os.walk(songs_path):
        dirs.sort()
        for file, i in zip(sorted(files), tqdm(range(num_files))):
            i = i + (counter-1)*100
            file_path = os.path.join(root, file)

            try:

                y, sr = librosa.load(os.fspath(file_path))

                mfccs19 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=57)
                mfccs20 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=67)
                mfccs21= librosa.feature.mfcc(y=y, sr=sr, n_mfcc=77)
                mfccs22 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=87)


                data_array[i,0] = np.mean(mfccs19)
                data_array[i,1] = np.var(mfccs19)
                data_array[i,2] = np.mean(mfccs20)
                data_array[i,3] = np.var(mfccs20)
                data_array[i,4] = np.mean(mfccs21)
                data_array[i,5] = np.var(mfccs21)
                data_array[i,6] = np.mean(mfccs22)
                data_array[i,7] = np.var(mfccs22)


            except:
                print(f'Problem file: {file_path}')
                for j in range(num_new_features):
                    data_array[i, j] = 0

        counter += 1

    return data_array


new_features_array = extract_new_features('/content/drive/MyDrive/genres_original') #audio file path

In [ ]:
df['mfccs19_mean'] = new_features_array[:,0]
df['mfccs19_var'] = new_features_array[:,1]
df['mfccs20_mean'] = new_features_array[:,2]
df['mfccs20_var'] = new_features_array[:,3]
df['mfccs21_mean'] = new_features_array[:,4]
df['mfccs21_var'] = new_features_array[:,5]
df['mfccs22_mean'] = new_features_array[:,6]
df['mfccs22_var'] = new_features_array[:,7]

In [ ]:
for i in range(-8,0,1):
    # Filter out the jazz genre except jazz.0054
    df.iloc[554,i] = df[ df.label == 'jazz'].iloc[np.r_[np.arange(0,54),np.arange(55,100)],i].mean()

df.to_csv('new_csv',index=False)
df

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,mfccs18_mean,mfccs18_var,mfccs19_mean,mfccs19_var,mfccs20_mean,mfccs20_var,mfccs21_mean,mfccs21_var,mfccs22_mean,mfccs22_var
0,blues,0.000000e+00,5.493504e-317,3.224717e-319,NaN,8.331307e-316,1.500302e-309,1.121529e-321,1.303754e-310,0.000000e+00,...,1.209538e-312,1.188318e-312,0.000000e+00,3.552144e-301,5.550625e-254,6.889868e+11,1.584952e-234,9.792529e-182,2.053165e-27,2.337439e-301
1,blues,3.552144e-301,5.833298e-302,2.485627e-297,3.508954e-292,6.999569e-283,9.878204e-278,5.832902e-302,5.843551e-302,6.914906e-310,...,1.209538e-312,1.018558e-312,3.322299e+257,2.348967e+251,1.237624e+224,5.274578e+170,2.440484e-152,7.514465e+199,5.030322e+180,1.039985e+267
2,blues,5.550625e-254,1.988085e-263,3.670230e-244,6.775656e-225,1.250861e-205,2.309227e-186,4.263085e-167,1.825277e-157,6.914906e-310,...,0.000000e+00,0.000000e+00,4.951208e+228,6.013470e-154,4.239752e+175,1.949024e+227,1.237624e+224,1.469171e+195,3.923884e+252,2.389547e+180
3,blues,6.889868e+11,1.560040e-239,2.336983e-301,8.585381e-254,1.462205e-128,1.271947e+31,2.877766e-220,2.339717e-301,6.914906e-310,...,1.060998e-312,0.000000e+00,-3.384607e+125,1.850193e-303,4.764181e-287,5.562983e-307,1.180296e+243,1.277347e-152,1.236064e-259,1.114601e+171
4,blues,1.584952e-234,2.699371e-109,2.348154e+50,5.308541e-201,2.353844e-301,2.927120e-215,2.140310e-80,4.334950e+69,6.914906e-310,...,1.039778e-312,1.103438e-312,6.013346e-154,1.577623e+214,1.337170e-152,4.443905e+252,4.873203e+199,6.611114e+149,2.848992e-310,8.344027e-309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,1.153055e+214,6.365737e+151,2.594592e+161,5.981834e-154,2.068343e+161,1.949489e+233,1.356156e+248,3.105709e+175,6.914906e-310,...,1.209538e-312,0.000000e+00,1.082218e-312,0.000000e+00,0.000000e+00,1.082218e-312,1.103438e-312,0.000000e+00,2.717361e-322,1.082218e-312
996,rock,1.391080e-258,4.652037e+151,2.382452e+180,4.240008e+228,1.590503e-258,4.652037e+151,2.382452e+180,4.240008e+228,6.914906e-310,...,1.124658e-312,1.082218e-312,1.209538e-312,0.000000e+00,0.000000e+00,1.039778e-312,1.124658e-312,0.000000e+00,0.000000e+00,1.082218e-312
997,rock,4.002475e+151,2.683674e-260,6.047611e-138,2.569679e+161,5.709514e+228,5.591219e+252,2.944113e-314,4.243992e-314,6.914906e-310,...,1.124658e-312,1.082218e-312,1.145878e-312,0.000000e+00,2.470328e-322,1.145878e-312,1.082218e-312,0.000000e+00,0.000000e+00,1.103438e-312
998,rock,1.761257e-312,1.999027e+289,1.996650e+289,3.393953e-111,3.962365e-133,4.852148e-308,1.333603e+241,1.979822e-310,6.914906e-310,...,0.000000e+00,0.000000e+00,1.018558e-312,0.000000e+00,0.000000e+00,1.018558e-312,1.145878e-312,0.000000e+00,2.470328e-322,1.060998e-312


In [ ]:
dp = pd.read_csv('/content/drive/MyDrive/dataset_6_csv')
dp.head()

,label,chroma_cens_mean,chroma_cens_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,tonnetz_mean,tonnetz_var,chroma_stft_mean,...,mfccs2_mean,mfccs2_var,mfccs3_mean,mfccs3_var,mfccs4_mean,mfccs4_var,mfccs5_mean,mfccs5_var,mfccs6_mean,mfccs6_var
0,blues,0.272263,0.009206,20.523289,76.801191,0.004496,0.000030,0.006427,0.004310,0.350129,...,0.729792,1201.604492,0.643018,919.671143,0.427209,743.078430,0.278182,623.940552,0.075146,538.383850
1,blues,0.261079,0.015171,20.672486,67.861845,0.002298,0.000024,0.029378,0.004747,0.340849,...,-0.191769,2427.744385,-0.095275,1839.843018,-0.447561,1480.468994,-0.362621,1239.792358,-0.517989,1067.379395
2,blues,0.262717,0.014313,22.191292,76.122598,0.002635,0.000011,0.024226,0.010467,0.363538,...,-0.242961,1260.250732,-0.848506,960.121521,-0.798550,779.632996,-0.909575,656.125000,-0.734296,566.632812
3,blues,0.226096,0.032214,21.423621,63.386763,0.000955,0.000003,0.026663,0.017942,0.404854,...,-0.917738,2388.502930,-1.725891,1800.508911,-2.034785,1447.093750,-2.367971,1211.789917,-2.301722,1042.670654
4,blues,0.259727,0.015875,21.459618,63.033892,0.003240,0.000019,-0.020944,0.010100,0.308526,...,-5.902440,1791.375977,-4.868073,1365.617676,-4.242023,1104.110596,-3.698091,931.020142,-3.299158,802.696533
